In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import openpyxl

# Inisialisasi aplikasi FastAPI
app = FastAPI()
# Nama file Excel
EXCEL_FILE = "products.xlsx"

# Model Pydantic untuk validasi input
class Product(BaseModel):
    name: str
    price: float
    description: str = None

# Fungsi untuk membaca data produk dari Excel
def read_products_from_excel():
    workbook = openpyxl.load_workbook(EXCEL_FILE)
    sheet = workbook.active
    products = {}
    for row in sheet.iter_rows(min_row=2, values_only=True):  # Lewati header
        product_id, name, price, description = row
        products[product_id] = {"name": name, "price": price, "description": description}
    return products

In [2]:
# Fungsi untuk menulis data produk ke Excel
def write_products_to_excel(products):
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.append(["ID", "Name", "Price", "Description"])  # Header
    for product_id, product_data in products.items():
        sheet.append([product_id, product_data["name"], product_data["price"], product_data.get("description", "")])
    workbook.save(EXCEL_FILE)

# Inisialisasi database dari Excel
try:
    fake_products_db = read_products_from_excel()
except FileNotFoundError:
    # Jika file Excel tidak ditemukan, buat file baru dengan data awal
    fake_products_db = {
        1: {"name": "Laptop", "price": 15000, "description": "Laptop gaming high-end"},
        2: {"name": "Smartphone", "price": 7000, "description": "Smartphone terbaru dengan kamera bagus"},
    }
    write_products_to_excel(fake_products_db)

In [3]:
# Endpoint untuk menambahkan produk baru
@app.post("/products/")
def create_product(product: Product):
    new_id = max(fake_products_db.keys()) + 1
    fake_products_db[new_id] = product.dict()
    write_products_to_excel(fake_products_db)  # Simpan ke Excel
    return {"id": new_id, "product": product}

# Endpoint untuk menampilkan semua produk
@app.get("/products/")
def get_products():
    return fake_products_db

# Endpoint untuk menampilkan produk berdasarkan ID
@app.get("/products/{product_id}")
def read_product(product_id: int):
    product = fake_products_db.get(product_id)
    if product is None:
        raise HTTPException(status_code=404, detail="Product not found")
    return product

In [4]:
# Endpoint untuk memperbarui produk berdasarkan ID
@app.put("/products/{product_id}")
def update_product(product_id: int, product: Product):
    if product_id not in fake_products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    fake_products_db[product_id] = product.dict()
    write_products_to_excel(fake_products_db)  # Simpan perubahan ke Excel
    return {"id": product_id, "product": product}

# Endpoint untuk menghapus produk berdasarkan ID
@app.delete("/products/{product_id}")
def delete_product(product_id: int):
    if product_id not in fake_products_db:
        raise HTTPException(status_code=404, detail="Product not found")
    del fake_products_db[product_id]
    write_products_to_excel(fake_products_db)  # Simpan perubahan ke Excel
    return {"message": f"Product {product_id} deleted successfully"}

# Jalankan server FastAPI menggunakan Uvicorn
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop